<a href="https://colab.research.google.com/github/ArmanFekri/Gee_Enviroment/blob/main/Sentinel2_Download_image.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import ee
import geemap
import folium
import json
import geopandas as gpd
import pandas as pd
import os

In [ ]:
# ee.Authenticate()
ee.Initialize()

# import shpfile and convert to ee.Geometry

In [ ]:
shape = gpd.read_file("/content/drive/MyDrive/Gee/shpfile/ROI.shp")
js = json.loads(shape.to_json())
geometry = ee.Geometry(ee.FeatureCollection(js).geometry())

# build image collection and mask clouds

In [ ]:
def maskS2clouds(image):
  qa = image.select('QA60');
  # Bits 10 and 11 are clouds and cirrus, respectively.
  cloudBitMask = 1 << 10;
  cirrusBitMask = 1 << 11;
  # Both flags should be set to zero, indicating clear conditions.
  mask = qa.bitwiseAnd(cloudBitMask).eq(0)\
      .And(qa.bitwiseAnd(cirrusBitMask).eq(0));

  return image.updateMask(mask)\
          .select('B.*')\
          .divide(10000)\
          .copyProperties(image, ["system:time_start"])

dataset = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')\
                  .filterBounds(geometry)\
                  .filterDate('2020-01-01', '2020-02-01')\
                  .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',20))\
                  .map(maskS2clouds)\
                  # .median()\
                  # .clip(geometry);

visualization = {'min': 0.0,'max': 0.3,'bands': ['B4', 'B3', 'B2']};

# indices

In [ ]:
def indexBands(img):
  NDVI = img.normalizedDifference(['B5', 'B4']).rename('ndvi')

  EVI = img.expression(
      '2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))',
      {
          'NIR': img.select('B5'),
          'RED': img.select('B4'),
          'BLUE': img.select('B2'),
      },
  ).rename('evi')
  return img.addBands([NDVI, EVI])

dataset = dataset.map(indexBands)

# check structure of dataset variable

In [ ]:
dataset.getInfo().keys()

dict_keys(['type', 'bands', 'id', 'version', 'properties', 'features'])

In [ ]:
len(dataset.getInfo()['features'])

8

In [ ]:
dataset.getInfo()['features'][0]

{'type': 'Image',
 'bands': [{'id': 'B1',
   'data_type': {'type': 'PixelType',
    'precision': 'float',
    'min': 0,
    'max': 6.553500175476074},
   'dimensions': [1830, 1830],
   'crs': 'EPSG:32639',
   'crs_transform': [60, 0, 399960, 0, -60, 4100040]},
  {'id': 'B2',
   'data_type': {'type': 'PixelType',
    'precision': 'float',
    'min': 0,
    'max': 6.553500175476074},
   'dimensions': [10980, 10980],
   'crs': 'EPSG:32639',
   'crs_transform': [10, 0, 399960, 0, -10, 4100040]},
  {'id': 'B3',
   'data_type': {'type': 'PixelType',
    'precision': 'float',
    'min': 0,
    'max': 6.553500175476074},
   'dimensions': [10980, 10980],
   'crs': 'EPSG:32639',
   'crs_transform': [10, 0, 399960, 0, -10, 4100040]},
  {'id': 'B4',
   'data_type': {'type': 'PixelType',
    'precision': 'float',
    'min': 0,
    'max': 6.553500175476074},
   'dimensions': [10980, 10980],
   'crs': 'EPSG:32639',
   'crs_transform': [10, 0, 399960, 0, -10, 4100040]},
  {'id': 'B5',
   'data_type': 

In [ ]:
dataset.getInfo()['features'][0]['properties']

{'system:time_start': 1578296367633,
 'system:index': '20200106T073209_20200106T073209_T39SVA'}

In [ ]:
list1 = []
for i in range(len(dataset.getInfo()['features'])):
  img_id = dataset.getInfo()['features'][i]['properties']['system:index']
  list1.append(img_id)
list1

['20200106T073209_20200106T073209_T39SVA',
 '20200108T072301_20200108T072255_T39SVA',
 '20200108T072301_20200108T072255_T39SVB',
 '20200123T072209_20200123T072246_T39SVB',
 '20200126T073059_20200126T073058_T39SVA',
 '20200126T073059_20200126T073058_T39SVB',
 '20200128T072141_20200128T072141_T39SVA',
 '20200128T072141_20200128T072141_T39SVB',
 '20200205T073009_20200205T073543_T39SVB',
 '20200207T072051_20200207T072046_T39SVA',
 '20200207T072051_20200207T072046_T39SVB',
 '20200227T071841_20200227T071839_T39SVB',
 '20200306T072749_20200306T073328_T39SVA',
 '20200311T072711_20200311T073308_T39SVA',
 '20200313T071659_20200313T072439_T39SVA',
 '20200331T072611_20200331T073205_T39SVB',
 '20200402T071619_20200402T072453_T39SVB',
 '20200405T072609_20200405T073256_T39SVB',
 '20200415T072609_20200415T073422_T39SVA',
 '20200415T072609_20200415T073422_T39SVB',
 '20200427T071621_20200427T072118_T39SVB']

In [ ]:
image_ids = dataset.aggregate_array('system:index').getInfo()
image_ids

['20200106T073209_20200106T073209_T39SVA',
 '20200108T072301_20200108T072255_T39SVA',
 '20200108T072301_20200108T072255_T39SVB',
 '20200123T072209_20200123T072246_T39SVB',
 '20200126T073059_20200126T073058_T39SVA',
 '20200126T073059_20200126T073058_T39SVB',
 '20200128T072141_20200128T072141_T39SVA',
 '20200128T072141_20200128T072141_T39SVB']

## * ee.aggregate_array: Aggregates over a given property of the objects in a collection, calculating a list of all the values of the selected property.

In [ ]:
# Export all image in image-collection
for i, img_id in enumerate(image_ids):
  image = ee.Image(dataset.filter(ee.Filter.eq('system:index', img_id)).first())
  task = ee.batch.Export.image.toDrive(**{
    'image':image.select(['B4', 'B3', 'B2']),
    'description':'img{}'.format(i+1),
    'folder': 'Gee',
    'scale':10,
    'region':geometry.getInfo()['coordinates'],
    'maxPixels': 1e13

    # task.start()
})

# median image from image-collection

In [ ]:
image = dataset.median().clip(geometry)

# Bands' names of image

In [ ]:
pd.DataFrame(image.getInfo()['bands'], columns=['id'])

,id
0,B1
1,B2
2,B3
3,B4
4,B5
5,B6
6,B7
7,B8
8,B8A
9,B9


# plot raster on map

In [ ]:
Map = geemap.Map()
Map.add_basemap('Esri.WorldImagery')
Map.centerObject(geometry)
Map.addLayer(image, visualization, 'Sentinel image')
Map

# Export image to Drive

In [ ]:
task = ee.batch.Export.image.toDrive(**{
    'image':image.select(['B4', 'B3', 'B2']),
    'description':'img',
    'folder': 'Gee',
    'scale':10,
    'region':geometry.getInfo()['coordinates'],
    'maxPixels': 1e13
})

# task.start()

# Export image to disk

In [ ]:
filename='/content/drive/MyDrive/Gee/test.tif'
# Exporting all bands as one single image
mage = image.clip(geometry).unmask()
geemap.ee_export_image(image, filename=filename, scale=10, region=geometry, file_per_band=False)


Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/Gee/test.tif


In [ ]:
# Exporting each band as one image
geemap.ee_export_image(
    image, filename=filename, scale=10, region=geometry, file_per_band=True
)

# Download image collection

In [ ]:
out_dir = '/content/drive/MyDrive/Gee'
geemap.ee_export_image_collection(dataset, out_dir=out_dir)